# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_csv = pd.read_csv('../output_data/cities.csv')
cities_csv.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,Rikitea,-23.12,-134.97,73.67,80,0,17.07,1603111965,PF
1,Bluff,-46.60,168.33,46.99,85,96,18.01,1603111965,NZ
2,Esperance,-33.87,121.90,60.33,84,0,15.12,1603111965,AU
3,Shouguang,36.88,118.74,61.00,50,0,7.05,1603111965,CN
4,Khorixas,-20.37,14.97,103.03,7,0,3.42,1603111966,NaN


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_csv[["Lat", "Lng"]].astype(float)
humidity_weight = cities_csv['Humidity'].astype(float)

# create figure
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_weight, 
                                 dissipating=False, max_intensity=200, point_radius=4)
fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# MY IDEAL WEATHER HAS CLOUDS!!!!!!! therefore...

# A max temperature lower than 80 degrees but higher than 50.
# Wind speed less than 15 mph.
# More than 10% cloudiness.


ideal_weather_df = cities_csv.copy()
ideal_weather_df = ideal_weather_df[(ideal_weather_df['Max Temp'] < 80) & (ideal_weather_df['Max Temp'] > 50)]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Wind Speed"] <= 15]
ideal_weather_df = ideal_weather_df[ideal_weather_df["Cloudiness"] >= 10]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = ideal_weather_df.copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
17,Flinders,-34.58,150.86,62.01,78,100,1.01,1603111969,AU,
22,Messíni,37.05,22.01,73.00,60,99,2.82,1603111970,GR,
28,Geraldton,-28.77,114.60,66.99,76,87,4.00,1603111971,AU,
41,Saint-Philippe,-21.36,55.77,78.01,68,32,5.99,1603111974,RE,
46,Torbay,47.67,-52.73,53.01,94,100,4.94,1603111928,CA,


In [12]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# happy testing small list of happy
hotel_df_mini = hotel_df.iloc[0:10]

error_number = 0 

for index, row in hotel_df.iterrows():
    
    coords = f"{hotel_df.Lat[index]},{hotel_df.Lng[index]}"
    city = hotel_df.City[index]

    params = {
        "location": coords, 
        "radius": "5000",
        "type": "lodging", 
        "key": g_key,
    }

    response = requests.get(base_url, params=params).json()
    try: 
        hotel_df.loc[index, "Hotel Name"] = response['results'][1]['name']
    except(KeyError, IndexError):
        error_number += 1


print(f"There were {error_number} errors")

There were 13 errors


In [13]:
hotel_df['Hotel Name'].replace("", np.nan, inplace=True)
hotel_df.dropna(how="any", inplace=True)

In [14]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
17,Flinders,-34.58,150.86,62.01,78,100,1.01,1603111969,AU,Shellharbour Beachside Holiday Park
22,Messíni,37.05,22.01,73.00,60,99,2.82,1603111970,GR,GMP Bouka Resort Hotel
28,Geraldton,-28.77,114.60,66.99,76,87,4.00,1603111971,AU,Hospitality Geraldton
41,Saint-Philippe,-21.36,55.77,78.01,68,32,5.99,1603111974,RE,Le four à pain
46,Torbay,47.67,-52.73,53.01,94,100,4.94,1603111928,CA,See The Sea
...,...,...,...,...,...,...,...,...,...,...
666,Ichhāwar,23.02,77.02,78.22,74,97,4.05,1603112962,IN,Hotel Dreem
669,Zhezkazgan,47.80,67.71,55.81,47,41,5.01,1603112962,KZ,Kosmos
675,Fleetwood,53.93,-3.01,55.00,82,100,8.99,1603112964,GB,Savoy Hotel
678,Alyangula,-13.85,136.42,79.57,74,57,9.04,1603112965,AU,SRA U Block containers


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
